In [6]:
import torch
from torchvision.models import resnet101, ResNet

b, k, c, h, w = 4, 8, 1024, 14, 14

In [2]:
a = torch.randn(b, k, h, w)
b = torch.randn(b, c, h, w)

In [ ]:
texts = ['back', 'beak', 'belly', 'breast', 'leg', 'tail', 'wing', 'throat']